In [ ]:
#!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-1wsuy16z/kobert-tokenizer_368aa220a51749b5b4a3a13a099a1a64
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-1wsuy16z/kobert-tokenizer_368aa220a51749b5b4a3a13a099a1a64
  Created wheel for kobert-tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4649 sha256=6f9e6a2f26bb9e3431c7595ac4097e07a14ff3ee21a9c8e4558aa5c69f725d89
  Stored in directory: /tmp/pip-ephem-wheel-cache-q54k6x6e/wheels/10/b4/d9/cb627bbfaefa266657b0b4e8127f7bf96d27376fa1a23897b4
Successfully built kobert-tokenizer


In [ ]:
!pip install ipywidgets  # for vscode
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-2fh2xgvp
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-2fh2xgvp
     |████████████████████████████████| 132 kB 5.3 MB/s 
     |████████████████████████████████| 344 kB 46.8 MB/s 
     |████████████████████████████████| 47.3 MB 60 kB/s 
     |████████████████████████████████| 4.5 MB 47.4 MB/s 
     |████████████████████████████████| 1.2 MB 41.8 MB/s 
     |████████████████████████████████| 4.0 MB 37.8 MB/s 
     |████████████████████████████████| 596 kB 28.6 MB/s 
     |████████████████████████████████| 880 kB 45.1 MB/s 
     |████████████████████████████████| 6.6 MB 39.4 MB/s 
     |████████████████████████████████| 77 kB 6.3 MB/s 
     |████████████████████████████████| 79 kB 7.7 MB/s 
     |████████████████████████████████| 8.7 MB 43.7 MB/s 
     |████████████████████████████████| 127 kB 51.1 MB/s 
  Created wheel for kobert: filename=kobe

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
from tqdm import tqdm_notebook
import pandas as pd
from sklearn.model_selection import train_test_split


In [ ]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
device = torch.device("cuda:0")
#device = torch.device("cpu")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset_train=pd.read_csv('/content/drive/Shareddrives/MLP/data/train_data.csv')
dataset_test=pd.read_csv('/content/drive/Shareddrives/MLP/data/test_data.csv')

nlp.data.TSVDataset함수를 이용하는 경우도 있는데 이것은 TSV는 탭으로 구분된 파일의 경우에 쓴다. 다만 read_csv로 만든 dataset은 리스트로 되어있지 않기때문에 한번 리스트로 바꿔줄 필요가 있다.

데이터 받기
dataset_train,datatset_test


In [ ]:
train, valid = train_test_split(dataset_train, test_size=0.2, random_state=42)

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
  ## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
dataset_list=[list(train.iloc[i]) for i in range(len(train))]
dataset_valid=[list(valid.iloc[i])for i in range(len(valid))]

In [ ]:
data_train = BERTDataset(dataset_list, 1, 2, tok, max_len, True, False)


In [ ]:
data_valid=BERTDataset(dataset_valid,1,2,tok,max_len,True,False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_valid, batch_size=batch_size, num_workers=2)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)


In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

RuntimeError: ignored

In [ ]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids,label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/571 [00:00<?, ?it/s]

 ====================== epoch 1 ======================


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


epoch 1 batch id 1 loss 1.9624756574630737 train acc 0.171875
 ====================== epoch 1 ======================
 ====================== epoch 1 ======================
 ====================== epoch 1 ======================
 ====================== epoch 1 ======================
 ====================== epoch 1 ======================
 ====================== epoch 1 ======================
 ====================== epoch 1 ======================
 ====================== epoch 1 ======================
 ====================== epoch 1 ======================
 ====================== epoch 1 ======================
 ====================== epoch 1 ======================
 ====================== epoch 1 ======================
 ====================== epoch 1 ======================
 ====================== epoch 1 ======================
 ====================== epoch 1 ======================
 ====================== epoch 1 ======================
 ====================== epoch 1 ======================
 ==

  0%|          | 0/571 [00:00<?, ?it/s]

...

  0%|          | 0/571 [00:00<?, ?it/s]

epoch 3 test acc 0.9555203284730991


  0%|          | 0/571 [00:00<?, ?it/s]

 ====================== epoch 4 ======================
epoch 4 batch id 1 loss 0.17408303916454315 train acc 0.9375
 ====================== epoch 4 ======================
 ====================== epoch 4 ======================
 ====================== epoch 4 ======================
 ====================== epoch 4 ======================
 ====================== epoch 4 ======================
 ====================== epoch 4 ======================
 ====================== epoch 4 ======================
 ====================== epoch 4 ======================
 ====================== epoch 4 ======================
 ====================== epoch 4 ======================
 ====================== epoch 4 ======================
 ====================== epoch 4 ======================
 ====================== epoch 4 ======================
 ====================== epoch 4 ======================
 ====================== epoch 4 ======================
 ====================== epoch 4 ======================
 ===

  0%|          | 0/571 [00:00<?, ?it/s]

In [ ]:
data_test=[list(dataset_test.iloc[i]) for i in range(len(dataset_test))]

In [ ]:
test_loader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

In [ ]:
result = []
model.eval()
with torch.no_grad():
    for batch_id, (token_ids, valid_length, segment_ids) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        result.append(model(token_ids, valid_length, segment_ids))

In [ ]:
result_ = []
for i in result:
    for j in i:
        result_.append(int(torch.argmax(j)))
        
out = [list(label_dict.keys())[_] for _ in result_]

submission["label"] = out

submission.to_csv("sample_submission.csv", index = False)

In [ ]:
submission.sample(3)